In [26]:
import os
import zipfile
import pandas as pd
import joblib
from dateutil import parser
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error, mean_absolute_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
import re

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# 1. Data Preparation

In [27]:
# time categories
def get_time_of_day(dt):
    hour = dt.hour
    if hour >=5 and hour < 8:
        return 'Early Morning'
    elif hour >= 8 and hour < 11:
        return 'Morning'
    elif hour >= 11 and hour < 14:
        return 'Midday'
    elif hour >= 14 and hour < 17:
        return 'Afternoon'
    elif hour >= 17 and hour < 20:
        return 'Evening'
    elif hour >= 20 and hour < 23:
        return 'Night'
    else:
        return 'Late Night'
 
# Remove UTC offset from datetime strings
def remove_utc_offset(datetime_str):
    dt = parser.parse(datetime_str)
    return dt.strftime('%Y-%m-%dT%H:%M:%S.%f')

# function to get features
def getfeatures(df):

    df['departuretime'] = df['segmentsDepartureTimeRaw'].apply(remove_utc_offset) 
    df['departuretime'] = pd.to_datetime(df['departuretime'], utc=False)
    
    # time category
    df['time_category'] = df['departuretime'].apply(get_time_of_day)
    
    # departure date
    df['date'] = (df['departuretime'] - pd.Timedelta(hours=2)).dt.date

    # no. of days from flight
    df['days_from_flight'] = (df['date'] - pd.to_datetime(df['searchDate']).dt.date)

    return df

def datefeatures(df):
    # Extract the year
    df['year'] = pd.to_datetime(df['date']).dt.year

    # Extract the month
    df['month'] = pd.to_datetime(df['date']).dt.month

    # Extract the day of the week (Monday=0, Sunday=6)
    df['day_of_week'] = pd.to_datetime(df['date']).dt.dayofweek

    # Extract the day of the month
    df['day_of_month'] = pd.to_datetime(df['date']).dt.day

    return df

def extract_days(duration_string):
    days = re.search(r'(\d+) days', duration_string)
    return int(days.group(1)) if days else None

In [8]:
# Root directory containing subfolders 
root_dir = '../raw/itineraries_csv'

dataframes = []

# Loop through each subfolder
for directory in os.listdir(root_dir):

  if os.path.isdir(os.path.join(root_dir, directory)) and not directory.startswith('.'):
    print(directory)
    folder_path = os.path.join(root_dir, directory)  

    # Loop through zip files
    for filename in os.listdir(folder_path):
      
      if filename.endswith('.zip'):
      
        # Full path of zip file
        zip_path = os.path.join(folder_path, filename)  
        
        # Extract zip contents
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
          zip_ref.extractall(folder_path)
              
          # Parse CSV file in zip 
          csv_path = os.path.join(folder_path, [x for x in zip_ref.namelist() if x.endswith('.csv')][0])      
          df = pd.read_csv(csv_path)
          
          # Apply processing 
          df = df[df['isNonStop'] == True]  
          df = getfeatures(df)
          df = datefeatures(df)

          # median fare
          df = df.groupby(['segmentsDepartureAirportCode', 'segmentsArrivalAirportCode', 'segmentsCabinCode', 'date', 'time_category', 'days_from_flight', 'year', 'month', 'day_of_week', 'day_of_month'])['totalFare'].median().reset_index(name='medianfare')
          
          # Append to list
          dataframes.append(df) 
        
# Combine dataframes
combined_df = pd.concat(dataframes, ignore_index=True)

combined_df.head()

ATL
BOS
CLT
DEN
DFW
DTW
EWR
IAD
JFK
LAX
LGA
MIA
OAK
ORD
PHL
SFO


,segmentsDepartureAirportCode,segmentsArrivalAirportCode,segmentsCabinCode,date,time_category,days_from_flight,year,month,day_of_week,day_of_month,medianfare
0,ATL,BOS,coach,2022-04-17,Afternoon,1 days,2022,4,6,17,398.6
1,ATL,BOS,coach,2022-04-17,Early Morning,1 days,2022,4,6,17,248.6
2,ATL,BOS,coach,2022-04-17,Evening,1 days,2022,4,6,17,398.6
3,ATL,BOS,coach,2022-04-17,Midday,1 days,2022,4,6,17,248.6
4,ATL,BOS,coach,2022-04-17,Morning,1 days,2022,4,6,17,248.6


In [30]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1649244 entries, 0 to 1649243
Data columns (total 11 columns):
 #   Column                        Non-Null Count    Dtype         
---  ------                        --------------    -----         
 0   segmentsDepartureAirportCode  1649244 non-null  object        
 1   segmentsArrivalAirportCode    1649244 non-null  object        
 2   segmentsCabinCode             1649244 non-null  object        
 3   date                          1649244 non-null  datetime64[ns]
 4   time_category                 1649244 non-null  object        
 5   days_from_flight              1649244 non-null  int64         
 6   year                          1649244 non-null  int32         
 7   month                         1649244 non-null  int32         
 8   day_of_week                   1649244 non-null  int32         
 9   day_of_month                  1649244 non-null  int32         
 10  medianfare                    1649244 non-null  float64       
dty

In [28]:
combined_df['days_from_flight'] = combined_df['days_from_flight'].astype('str')
combined_df['days_from_flight'] = combined_df['days_from_flight'].apply(extract_days)

In [37]:
len(combined_df)

1649244

In [9]:
combined_df.to_csv('medianfares.csv', index=False)

In [18]:
combined_df = pd.read_csv('medianfares.csv')

In [38]:
print('Earliest date:', combined_df['date'].min())
print('Latest date:', combined_df['date'].max())

Earliest date: 2022-04-16 00:00:00
Latest date: 2022-07-18 00:00:00


In [39]:
# Define the split date
split_date = pd.Timestamp('2022-06-17')
combined_df['date'] = pd.to_datetime(combined_df['date'])

# Create the train and test sets
train = combined_df.loc[combined_df.date < split_date]
test = combined_df.loc[combined_df.date >= split_date]

print(len(train))
print(len(test))

1321239
328005


In [40]:
train.head()

,segmentsDepartureAirportCode,segmentsArrivalAirportCode,segmentsCabinCode,date,time_category,days_from_flight,year,month,day_of_week,day_of_month,medianfare
0,ATL,BOS,coach,2022-04-17,Afternoon,1,2022,4,6,17,398.6
1,ATL,BOS,coach,2022-04-17,Early Morning,1,2022,4,6,17,248.6
2,ATL,BOS,coach,2022-04-17,Evening,1,2022,4,6,17,398.6
3,ATL,BOS,coach,2022-04-17,Midday,1,2022,4,6,17,248.6
4,ATL,BOS,coach,2022-04-17,Morning,1,2022,4,6,17,248.6


# 2. Modelling

In [48]:
def fitmodel(model, model_name, train=train, test=test):

    X_train = train.drop(columns='medianfare')
    y_train = train['medianfare']

    X_test = test.drop(columns='medianfare')
    y_test = test['medianfare']

    # Define categorical and numeric columns for preprocessing
    categorical_cols = ['segmentsDepartureAirportCode', 'segmentsArrivalAirportCode', 'segmentsCabinCode', 'time_category']
    numeric_cols = ['year',  'month', 'day_of_week', 'day_of_month', 'days_from_flight']

    # Preprocessing and modeling pipeline
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', 'passthrough', numeric_cols), 
            ('cat', OneHotEncoder(), categorical_cols)
        ])

    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])

    # Train the model
    pipeline.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = pipeline.predict(X_test)

    # Assuming you have a scikit-learn pipeline named 'pipeline'
    joblib.dump(pipeline, f'../models/{model_name}.pkl')

    # Evaluate the model
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred)

    return pipeline, model_name, rmse, r2, mae, mape

In [49]:
cat = fitmodel(CatBoostRegressor(), 'median')

Learning rate set to 0.127437
0:	learn: 178.8832767	total: 133ms	remaining: 2m 12s
1:	learn: 170.1752199	total: 240ms	remaining: 1m 59s
2:	learn: 163.2071162	total: 340ms	remaining: 1m 52s
3:	learn: 157.5796433	total: 441ms	remaining: 1m 49s
4:	learn: 153.1336637	total: 543ms	remaining: 1m 47s
5:	learn: 149.5677644	total: 634ms	remaining: 1m 45s
6:	learn: 146.8039718	total: 721ms	remaining: 1m 42s
7:	learn: 144.7083838	total: 814ms	remaining: 1m 40s
8:	learn: 142.4865049	total: 896ms	remaining: 1m 38s
9:	learn: 140.9928122	total: 984ms	remaining: 1m 37s
10:	learn: 139.5119752	total: 1.07s	remaining: 1m 36s
11:	learn: 138.3585977	total: 1.17s	remaining: 1m 36s
12:	learn: 136.9163539	total: 1.27s	remaining: 1m 36s
13:	learn: 136.0611264	total: 1.38s	remaining: 1m 37s
14:	learn: 134.8188465	total: 1.48s	remaining: 1m 37s
15:	learn: 134.0995754	total: 1.57s	remaining: 1m 36s
16:	learn: 132.9722879	total: 1.69s	remaining: 1m 37s
17:	learn: 132.1353302	total: 1.81s	remaining: 1m 38s
18:	lear

In [50]:
print(cat)

(Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', 'passthrough',
                                                  ['year', 'month',
                                                   'day_of_week',
                                                   'day_of_month',
                                                   'days_from_flight']),
                                                 ('cat', OneHotEncoder(),
                                                  ['segmentsDepartureAirportCode',
                                                   'segmentsArrivalAirportCode',
                                                   'segmentsCabinCode',
                                                   'time_category'])])),
                ('regressor',
                 <catboost.core.CatBoostRegressor object at 0x000001EE5DF14F90>)]), 'median', 89.71553864260115, 0.6883966201259524, 62.62714866452494, 0.2113440630923253)


In [47]:
xgboost = fitmodel(XGBRegressor(), 'median2')

print(xgboost)

C:\Users\dlatl\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):


(Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', 'passthrough',
                                                  ['year', 'month',
                                                   'day_of_week',
                                                   'day_of_month',
                                                   'days_from_flight']),
                                                 ('cat', OneHotEncoder(),
                                                  ['segmentsDepartureAirportCode',
                                                   'segmentsArrivalAirportCode',
                                                   'segmentsCabinCode',
                                                   'time_category'])])),
                ('regressor',
                 XGBRegressor(base_score=None, booster=None, callbacks=None,
                              cols...
                              feature_types=None, gamma=None, grow_policy=None,
            

In [48]:
rforest = fitmodel(RandomForestRegressor(), 'median3')

print(rforest)

(Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', 'passthrough',
                                                  ['year', 'month',
                                                   'day_of_week',
                                                   'day_of_month',
                                                   'days_from_flight']),
                                                 ('cat', OneHotEncoder(),
                                                  ['segmentsDepartureAirportCode',
                                                   'segmentsArrivalAirportCode',
                                                   'segmentsCabinCode',
                                                   'time_category'])])),
                ('regressor', RandomForestRegressor())]), 'rf', 98.6780596513176, 0.6230289283508642, 64.68049112683988, 0.21106128587494188)


In [8]:
import json

unique_segments = combined_df[['segmentsDepartureAirportCode', 'segmentsArrivalAirportCode']].drop_duplicates()

# Create a dictionary where each departure airport has a list of arrival airports
airport_dict = {}
for index, row in unique_segments.iterrows():
    dep_airport = row['segmentsDepartureAirportCode']
    arr_airport = row['segmentsArrivalAirportCode']
    if dep_airport in airport_dict:
        airport_dict[dep_airport].append(arr_airport)
    else:
        airport_dict[dep_airport] = [arr_airport]

# Convert dictionary to JSON
json_output = json.dumps(airport_dict, indent=4)

# Printing or using the json_output as needed
print(json_output)


{
    "OAK": [
        "DEN",
        "LAX",
        "PHL",
        "DTW",
        "ORD"
    ],
    "IAD": [
        "SFO",
        "ATL",
        "BOS",
        "CLT",
        "DEN",
        "DFW",
        "DTW",
        "EWR",
        "JFK",
        "LAX",
        "LGA",
        "MIA",
        "ORD"
    ],
    "DEN": [
        "ATL",
        "BOS",
        "CLT",
        "DFW",
        "DTW",
        "EWR",
        "IAD",
        "JFK",
        "LAX",
        "LGA",
        "MIA",
        "OAK",
        "ONT",
        "ORD",
        "PHL",
        "SFO"
    ],
    "LGA": [
        "ATL",
        "BOS",
        "CLT",
        "DEN",
        "DFW",
        "DTW",
        "IAD",
        "MIA",
        "ORD",
        "LAX"
    ],
    "LAX": [
        "ATL",
        "BOS",
        "CLT",
        "DAL",
        "DEN",
        "DFW",
        "DTW",
        "EWR",
        "IAD",
        "JFK",
        "LGA",
        "MIA",
        "OAK",
        "ORD",
        "PHL",
        "SFO"
    ],
   

In [39]:
with open('flightroutes.json', 'w') as file:
    file.write(json_output)

# 3. Catboost with Distance/Duration Features

In [11]:
# Getting all Data

# Root directory containing subfolders 
root_dir = 'itineraries_csv'

dataframes = []

# Loop through each subfolder
for directory in os.listdir(root_dir):

  if os.path.isdir(os.path.join(root_dir, directory)) and not directory.startswith('.'):
    print(directory)
    folder_path = os.path.join(root_dir, directory)  

    # Loop through zip files
    for filename in os.listdir(folder_path):
      
      if filename.endswith('.zip'):
      
        # Full path of zip file
        zip_path = os.path.join(folder_path, filename)  
        
        # Extract zip contents
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
          zip_ref.extractall(folder_path)
              
          # Parse CSV file in zip 
          csv_path = os.path.join(folder_path, [x for x in zip_ref.namelist() if x.endswith('.csv')][0])      
          df = pd.read_csv(csv_path)
          
          # Apply processing 
          df = df[df['isNonStop'] == True]  
          df = getfeatures(df)
          df = datefeatures(df)

          # Append to list
          dataframes.append(df) 
        
# Combine dataframes
all_single = pd.concat(dataframes, ignore_index=True)
all_single.head()

OAK
DEN
LGA
LAX
ATL
CLT
PHL
DTW
IAD
JFK
DFW
BOS
EWR
SFO
ORD
MIA


,legId,searchDate,flightDate,startingAirport,destinationAirport,travelDuration,isBasicEconomy,isRefundable,isNonStop,totalFare,totalTravelDistance,segmentsDepartureTimeEpochSeconds,segmentsDepartureTimeRaw,segmentsArrivalTimeEpochSeconds,segmentsArrivalTimeRaw,segmentsArrivalAirportCode,segmentsDepartureAirportCode,segmentsAirlineName,segmentsAirlineCode,segmentsEquipmentDescription,segmentsDurationInSeconds,segmentsDistance,segmentsCabinCode,departuretime,time_category,date,days_from_flight
0,ce6c79e893312e42ddd97e22637ee980,2022-04-19,2022-05-20,OAK,DEN,PT2H33M,False,False,True,83.98,943.0,1653098280,2022-05-20T18:58:00.000-07:00,1653107460,2022-05-20T22:31:00.000-06:00,DEN,OAK,Frontier Airlines,F9,NaN,9180,943,coach,2022-05-20 18:58:00,Evening,2022-05-20,31
1,a0bc7fa432fd2475c3dbe17b376b1830,2022-04-19,2022-05-20,OAK,LAX,PT1H22M,False,False,True,42.79,NaN,1653062160,2022-05-20T08:56:00.000-07:00,1653067080,2022-05-20T10:18:00.000-07:00,LAX,OAK,Spirit Airlines,NK,NaN,4920,None,coach,2022-05-20 08:56:00,Morning,2022-05-20,31
2,3e224e69b09b19ba9a2562b55e275713,2022-04-19,2022-05-20,OAK,LAX,PT1H22M,False,False,True,66.97,NaN,1653051900,2022-05-20T06:05:00.000-07:00,1653056820,2022-05-20T07:27:00.000-07:00,LAX,OAK,Spirit Airlines,NK,AIRBUS INDUSTRIE A320 SHARKLETS,4920,None,coach,2022-05-20 06:05:00,Early Morning,2022-05-20,31
3,d89f49d3dd5a77fec2fabbcc258ef0f3,2022-04-19,2022-05-20,OAK,LAX,PT1H24M,False,False,True,133.60,338.0,1653056160,2022-05-20T07:16:00.000-07:00,1653061200,2022-05-20T08:40:00.000-07:00,LAX,OAK,Delta,DL,Embraer 175 (Enhanced Winglets),5040,338,coach,2022-05-20 07:16:00,Early Morning,2022-05-20,31
4,b22bb0ae76e6f92840354d33d614b922,2022-04-19,2022-05-20,OAK,LAX,PT1H21M,False,False,True,168.61,338.0,1653087840,2022-05-20T16:04:00.000-07:00,1653092700,2022-05-20T17:25:00.000-07:00,LAX,OAK,Delta,DL,Embraer 175 (Enhanced Winglets),4860,338,coach,2022-05-20 16:04:00,Afternoon,2022-05-20,31


In [25]:
# Duration and Distance Features

median_duration = all_single.groupby(['segmentsDepartureAirportCode','segmentsArrivalAirportCode', 'time_category'])['segmentsDurationInSeconds'].median().reset_index(name='median_duration')

median_distance = all_single[all_single['segmentsDistance'] != 'None']
median_distance = median_distance.groupby(['segmentsDepartureAirportCode','segmentsArrivalAirportCode', 'time_category'])['segmentsDistance'].median().reset_index(name='median_distance')

combined_df = pd.merge(combined_df, median_duration, how='left', on=['segmentsDepartureAirportCode', 'segmentsArrivalAirportCode', 'time_category'])
combined_df = pd.merge(combined_df, median_distance, how='left', on=['segmentsDepartureAirportCode', 'segmentsArrivalAirportCode', 'time_category'])

combined_df.head()

,segmentsDepartureAirportCode,segmentsArrivalAirportCode,segmentsCabinCode,date,time_category,days_from_flight,medianfare,year,month,day_of_week,day_of_month
0,OAK,DEN,coach,2022-05-20,Evening,31,83.980,2022,5,4,20
1,OAK,DEN,coach,2022-05-21,Evening,32,43.980,2022,5,5,21
2,OAK,DEN,coach,2022-05-22,Evening,33,173.980,2022,5,6,22
3,OAK,LAX,coach,2022-05-20,Afternoon,31,168.610,2022,5,4,20
4,OAK,LAX,coach,2022-05-20,Early Morning,31,100.285,2022,5,4,20


In [31]:
# Define the split date
split_date = pd.Timestamp('2022-06-17')
combined_df['date'] = pd.to_datetime(combined_df['date'])

# Create the train and test sets
train = combined_df.loc[combined_df.date < split_date]
test = combined_df.loc[combined_df.date >= split_date]

print(len(train))
print(len(test))

1321239
328005


In [35]:
# Define categorical and numeric columns for preprocessing
categorical_cols = ['segmentsDepartureAirportCode', 'segmentsArrivalAirportCode', 'segmentsCabinCode', 'time_category']
numeric_cols = ['year',  'month', 'day_of_week', 'day_of_month', 'days_from_flight', 'median_duration', 'median_distance']

def fitmodel2(model, model_name, train=train, test=test):

    X_train = train.drop(columns='medianfare')
    y_train = train['medianfare']

    X_test = test.drop(columns='medianfare')
    y_test = test['medianfare']

    # Preprocessing and modeling pipeline
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', 'passthrough', numeric_cols), 
            ('cat', OneHotEncoder(), categorical_cols)
        ])

    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])

    # Train the model
    pipeline.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = pipeline.predict(X_test)

    # Assuming you have a scikit-learn pipeline named 'pipeline'
    joblib.dump(pipeline, f'{model_name}.pkl')

    # Evaluate the model
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred)

    return pipeline, model_name, rmse, r2, mae, mape

In [36]:
cat2 = fitmodel2(CatBoostRegressor(), 'catboost2')
print(cat2)

Learning rate set to 0.127437
0:	learn: 176.4885393	total: 163ms	remaining: 2m 42s
1:	learn: 165.6213647	total: 213ms	remaining: 1m 46s
2:	learn: 156.7693525	total: 263ms	remaining: 1m 27s
3:	learn: 149.5701692	total: 312ms	remaining: 1m 17s
4:	learn: 143.7220964	total: 373ms	remaining: 1m 14s
5:	learn: 139.0529583	total: 437ms	remaining: 1m 12s
6:	learn: 135.1760285	total: 488ms	remaining: 1m 9s
7:	learn: 132.0802373	total: 544ms	remaining: 1m 7s
8:	learn: 129.5849659	total: 595ms	remaining: 1m 5s
9:	learn: 127.4678092	total: 656ms	remaining: 1m 4s
10:	learn: 125.7560730	total: 712ms	remaining: 1m 3s
11:	learn: 124.2226336	total: 762ms	remaining: 1m 2s
12:	learn: 122.8085946	total: 826ms	remaining: 1m 2s
13:	learn: 121.6664481	total: 892ms	remaining: 1m 2s
14:	learn: 120.6415272	total: 950ms	remaining: 1m 2s
15:	learn: 119.7708667	total: 1s	remaining: 1m 1s
16:	learn: 119.0111153	total: 1.05s	remaining: 1m 1s
17:	learn: 118.3005316	total: 1.12s	remaining: 1m 1s
18:	learn: 117.6003735	